# AA4: Fun with Window Functions

* Assigned: Nov 8
* Due: Nov 15, 10AM
* Extra credit: up to 3.75%

In this assignment you'll take your data science skills to the next level, learning how to use window functions to perform advanced SELECT queries.

#### Copy the AA4 folder into your Instabase repo before starting this assignment.

## Preparation

First, make sure that you fully understand how to use SQL concepts previously taught in class like aggregations, subqueries etc., as you will also need to use them.

Then, read [this tutorial](https://www.postgresql.org/docs/9.3/static/tutorial-window.html) to understand window functions. Note that they are not PostgreSQL-specific, and some might refer to them as "Analytical Functions" instead.

** Reference: **
* [Syntax](https://www.postgresql.org/docs/9.3/static/sql-expressions.html#SYNTAX-WINDOW-FUNCTIONS)
* [List of Aggregate Functions](https://www.postgresql.org/docs/9.3/static/functions-aggregate.html) (every aggregate function can also be used as a window function)
* [List of Window Functions](https://www.postgresql.org/docs/9.3/static/functions-window.html)


## Token Setup

In [140]:
# Your columbia uni that is used in SSOL
#
# IMPORTANT:  make sure this is consistent with the uni/alias used as your @columbia.edu email in SSOL
#
UNI = "wt2271"

# your instabase username (if you go to the instabase homepage, your username should be in the URL)
USER = "wesley"

# your repository name containing this assignment
REPO = "my-repo"

print "Provided UNI is {0}. Make sure it is correct.".format(UNI)


Provided UNI is wt2271. Make sure it is correct.


## Database Connection

Run following command to connect to database. You should see "Connected to ..." for successful connection.

In [2]:
# connect to db in public repo
ib.connect_db('ib://ewu/w4111-public/databases/w4111')

Connected to: ib://ewu/w4111-public/databases/w4111


### Jupyter Notes: _Read these carefully_

* You **may** create new IPython notebook cells to use for e.g. testing, debugging, exploring, etc.- this is encouraged in fact!- **just make sure that you run the final cell to submit your results**
  * you can press shift+enter to execute to code in the cell that your cursor is in.
* When you see `In [*]:` to the left of the cell you are executing, this means that the code / query is _running_. Please wait for the execution to complete
    * **If the cell is hanging- i.e. running for too long: you can restart the kernel**
    * To restart kernel using the menu bar: "Kernel >> Restart >> Clear all outputs & restart"), then re-execute cells from the top
* _Have fun!_

# Warm Up

**STOP!** Make sure you have read the [tutorial](https://www.postgresql.org/docs/9.3/static/tutorial-window.html) mentioned above before proceeding. 

You can imagine window functions as special functions that, for every row, will compute some value by looking at other rows as well. These other rows are the "window", hence the name.

An expression will have the following format:

```
some_function( args ... ) OVER ( <WINDOW_DEFINITION> )
```

The "window definition" is the key on how the function will look at other rows. It has three **optional** parts:

```
<WINDOW_DEFINITION> =
    [<PARTITIONING>] [<ORDERING>] [<FRAME_CLAUSE>]
```

- `<PARTITIONING>` defines how to split windows based on some column(s) or even expression(s);
- `<ORDERING>` defines how the rows should be ordered inside a window;
- `<FRAME_CLAUSE>` defines how rows that come before and after the current one should be included in the window or not. Intuitively, it has no meaning if you are not using an ordering. **The default (when ordering is set) is to include every row between the first one in the window and the current one.**

&nbsp;

Now, consider this sample table containing the number of cars that entered a shopping mall, breaking down by entrance and date.


In [11]:
%%sql
select * from fact_parking_entrances


,day,entrance,number_of_cars
0,2018-02-01,A,222
1,2018-02-02,A,122
2,2018-02-03,A,203
3,2018-02-04,A,474
4,2018-02-05,A,383
5,2018-02-06,A,292
6,2018-02-01,B,129
7,2018-02-02,B,178
8,2018-02-03,B,156
9,2018-02-04,B,381


&nbsp;

**Now let's see some window functions in practice. Feel free to play with them!**

In [12]:
%%sql
SELECT
    day,
    entrance,
    number_of_cars,
    
    --" Total # of cars over all entrances for the same day. "
    sum(number_of_cars) OVER (PARTITION BY day)
        as daily_sum_all_entrances,
        
    --" Look how the record's number_of_cars attribute can be divided"
    --" by the SUM() from above to calculate the percent of this record's"
    --" to the day's total. "
    number_of_cars*100.0 /
        ( sum(number_of_cars) OVER (PARTITION BY day) )
        as daily_entrance_percentage,
    
    --" How about defining the position in which each entrance stands if  "
    --"  we rank them by the number of cars on a specific day "
    rank() OVER (PARTITION BY day ORDER BY number_of_cars DESC)
        as daily_rank
        
FROM fact_parking_entrances
ORDER BY day, entrance


,day,entrance,number_of_cars,daily_sum_all_entrances,daily_entrance_percentage,daily_rank
0,2018-02-01,A,222,351,63.2478632478632479,1
1,2018-02-01,B,129,351,36.7521367521367521,2
2,2018-02-02,A,122,300,40.6666666666666667,2
3,2018-02-02,B,178,300,59.3333333333333333,1
4,2018-02-03,A,203,359,56.5459610027855153,1
5,2018-02-03,B,156,359,43.4540389972144847,2
6,2018-02-04,A,474,855,55.4385964912280702,1
7,2018-02-04,B,381,855,44.5614035087719298,2
8,2018-02-05,A,383,616,62.1753246753246753,1
9,2018-02-05,B,233,616,37.8246753246753247,2


In [13]:
%%sql
SELECT
    day,
    entrance,
    number_of_cars,
    
    --" Let's do a CUMULATIVE sum per entrance over days. "
    --" This SUM only differs from the above query's in the ORDER BY day clause"
    --" Note: Remember that the default frame clause is between the first row"
    --"  and the current record, so we omit it here. "
    sum(number_of_cars) OVER (PARTITION BY entrance ORDER BY day)
        as cum_sum_by_entrance,

    --" Same as the previous SUM expression, but we make the frame clause explicit"
    sum(number_of_cars) OVER (
            PARTITION BY entrance 
            ORDER BY day
            ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)
        as cum_sum_by_entrance2,
        
    --" Now, let's say that we want to get the max value per entrance, but only 
    --" between the previous day, the current one, and the next one."
    max(number_of_cars) OVER (
            PARTITION BY entrance
            ORDER BY day
            ROWS BETWEEN 1 PRECEDING AND 1 FOLLOWING
        ) as max_yday_today_tmrw_by_entrance
        
FROM fact_parking_entrances
ORDER BY entrance, day


,day,entrance,number_of_cars,cum_sum_by_entrance,cum_sum_by_entrance2,max_yday_today_tmrw_by_entrance
0,2018-02-01,A,222,222,222,222
1,2018-02-02,A,122,344,344,222
2,2018-02-03,A,203,547,547,474
3,2018-02-04,A,474,1021,1021,474
4,2018-02-05,A,383,1404,1404,474
5,2018-02-06,A,292,1696,1696,383
6,2018-02-01,B,129,129,129,178
7,2018-02-02,B,178,307,307,178
8,2018-02-03,B,156,463,463,381
9,2018-02-04,B,381,844,844,381


# "The interview"

You are being interviewed for a data scientist position at a major online store, and to get the job, you need to prove that you can answer some of their questions regarding past sales.

To do so, you have access to a dataset containing every sale between 2014 and 2017, stored in a single table. For simplicity, every transaction contains only one item.

The columns are the following:
* **transaction_id** - unique integer id for that transaction **(do not assume any ordering!)**
* **transaction_datetime** - date and time of the transaction, encoded as Postgres' timestamp type
* **customer_id** - unique integer id for that customer
* **customer_first_name**
* **customer_last_name**
* **shipping_state**
* **item_id** - unique integer id for that item
* **item_description**
* **item_price**

You may run this query to have some ideas on the schema you are dealing with:

In [76]:
%%sql
select * from fact_sales limit 5;

,transaction_id,transaction_datetime,customer_id,customer_first_name,customer_last_name,shipping_state,item_id,item_description,item_price
0,1,2014-01-01 00:06:40,100000236,Carol,Blake,Missouri,8068199275,Slacks,87.7799988
1,2,2014-01-01 00:29:26,300000427,Dominic,Welch,Michigan,2695940506,Sweater,59.4000015
2,3,2014-01-01 00:53:39,100000197,Blake,Ross,Delaware,1336619171,Slacks,63.8400002
3,4,2014-01-01 01:41:48,100000116,Alison,Johnston,Delaware,5187373849,Short Slip,12.8699999
4,5,2014-01-01 02:31:54,100000007,Dorothy,Thomson,Alabama,8586814459,Gown & Robe Set,65.6399994


Now, please write the SQL query for each of the following questions. 

** You must use window functions to answer each one. **

** You may need to check the list of available window functions listed in the Preparation section.**

Here's a list of some useful expressions for timestamp/datetime:

```
extract(year from some_timestamp_column) -> gets year from datetime
extract(month from some_timestamp_column) -> gets month from datetime
cast(some_timestamp_column as date) -> gets full date (year, month and day) from datetime
```


### Q1
__For each of the customers listed below by ID, show their individual cumulative sum over transactions made in 2015.__

Customer IDs: 300000073, 100000382, 100000148

You must provide a single query, containing the following columns:
* transaction_id
* transaction_datetime
* customer_id
* customer_last_name
* item_id
* item_description
* item_price
* cumulative_sum, containing the cumulative sum over time for such customer

In [16]:
%%sql
-- Complete your query here
select transaction_id, transaction_datetime,customer_id, customer_last_name, item_id,
item_description, item_price,
count(transaction_id)  over(partition by customer_id order by transaction_datetime) as cumulative_sum
from fact_sales 
where customer_id in (300000073, 100000382, 100000148) and extract(year from transaction_datetime)=2015

,transaction_id,transaction_datetime,customer_id,customer_last_name,item_id,item_description,item_price,cumulative_sum
0,18660,2015-06-30 06:54:59,100000148,Ogden,4398051135,Bra,6,1
1,19175,2015-07-15 00:09:50,100000148,Ogden,9862518267,Socks,10.8400002,2
2,21535,2015-09-22 04:58:19,100000148,Ogden,1625450539,Dress,54.8899994,3
3,13792,2015-02-10 01:05:43,100000382,Chapman,6388591017,Baby Quilt,100.68,1
4,17567,2015-05-29 12:49:20,100000382,Chapman,5310615417,Tie,41.7900009,2
5,19436,2015-07-22 18:55:48,100000382,Chapman,9873264896,Hat,10.8900003,3
6,13698,2015-02-07 07:02:07,300000073,White,5153370805,Shorts,43.9599991,1
7,13824,2015-02-10 19:51:00,300000073,White,5153370805,Shorts,43.9599991,2
8,14156,2015-02-20 20:46:16,300000073,White,8675740287,Baby Quilt,47.7599983,3
9,14473,2015-03-01 23:16:08,300000073,White,3123824581,Sweatpants,43.8899994,4


In [134]:
# submit query in string format
q1 = {
    "query": """
    select transaction_id, transaction_datetime,customer_id, customer_last_name, item_id,
item_description, item_price,
count(transaction_id)  over(partition by customer_id order by transaction_datetime) as cumulative_sum
from fact_sales 
where customer_id in (300000073, 100000382, 100000148) and extract(year from transaction_datetime)=2015
    """
}

### Q2

__What were the top 5 shipping states per year, based on the total value of transactions, for 2014 and 2015?__

You must provide a single query, containing the following columns
* year, string
* shipping_state, string
* total_sales, containing the total value of transactions
* position, containing the position of such item among the top 5 for such state.

**Hints:** 
* Use the [WITH clause](https://www.postgresql.org/docs/9.1/static/queries-with.html) to prep temporary tables 
* First calculate the total value of transactions per state, year pair
* Order the states in each year by total transaction value and compute their rank()
* Find the states in years 2014 and 215 with the appropriate ranks

In [66]:
%%sql
-- Complete your query here
with temp as (
select extract(year from transaction_datetime) as year,shipping_state, sum(item_price) as total_sales
from fact_sales 
group by year, shipping_state)

select * from (
select year, shipping_state, total_sales, rank() over (partition by year order by total_sales desc) as position 
from  temp
where year in (2014,2015)) as temp1
where position <6




,year,shipping_state,total_sales,position
0,2014,Texas,28979.7188,1
1,2014,Massachusetts,26338.0781,2
2,2014,Rhode Island,19873.248,3
3,2014,Arkansas,19846.1035,4
4,2014,Kentucky,19607.2676,5
5,2015,Texas,28772.832,1
6,2015,Massachusetts,27991.8965,2
7,2015,Kentucky,22268.5117,3
8,2015,Rhode Island,20617.5156,4
9,2015,Arkansas,17653.7949,5


In [135]:
# submit query in string format
q2 = {
    "query": """
    with temp as (
select extract(year from transaction_datetime) as year,shipping_state, sum(item_price) as total_sales
from fact_sales 
group by year, shipping_state)

select * from (
select year, shipping_state, total_sales, rank() over (partition by year order by total_sales desc) as position 
from  temp
where year in (2014,2015)) as temp1
where position <6

    """
}

### Q3

__Compute the daily total value of sales from 2014-01-01 to 2014-02-15 (inclusive), along with a running 7d average and a month-to-date total__

You must provide a single query, containing the following columns:
* date
* total_sales, containing the total for such date
* sales_7d_avg, containing the running 7d average (i.e. between 6 days before and the current date). Note: the average should be computed even if there's some missing data for the past 6 days (e.g. 2014-01-01).
* month_to_date, containing the cumulative sum from the start of the month to the current date.

**Hints:** 
* Check the [syntax reference](https://www.postgresql.org/docs/9.3/static/sql-expressions.html#SYNTAX-WINDOW-FUNCTIONS) on how to define a frame clause for a running avg.
* Use WITH to compute daily totals first

In [87]:
%%sql
-- Complete your query here
with temp as (select cast(transaction_datetime as date) as date, sum(item_price) as total_sales
from fact_sales 
group by date
order by date)

select date, total_sales, avg(total_sales) over (partition by extract(month from date) order by date 
                                                 rows between 6 preceding and
                                                current row) as sales_7d_avg ,
sum(total_sales) over(partition by extract(month from date) order by date) as month_to_date
from temp 
where date > '2014-01-01'::date and date <='2014-02-05'::date

,date,total_sales,sales_7d_avg,month_to_date
0,2014-01-02,1933.42004,1933.4200439453125,1933.42004
1,2014-01-03,1801.95007,1867.68505859375,3735.37012
2,2014-01-04,1061.10999,1598.82670084635424,4796.47998
3,2014-01-05,2227.86011,1756.08505249023438,7024.33984
4,2014-01-06,2297.97998,1864.46403808593755,9322.32031
5,2014-01-07,1919.49023,1873.63507080078125,11241.8105
6,2014-01-08,1796,1862.54434640066961,13037.8105
7,2014-01-09,1036.31006,1734.38577706473211,14074.1211
8,2014-01-10,1709.91003,1721.23720005580367,15784.0312
9,2014-01-11,1699.62,1812.45291573660711,17483.6504


In [136]:
# submit query in string format
q3 = {
    "query": """
    with temp as (select cast(transaction_datetime as date) as date, sum(item_price) as total_sales
from fact_sales 
group by date
order by date)

select date, total_sales, avg(total_sales) over (partition by extract(month from date) order by date 
                                                 rows between 6 preceding and
                                                current row) as sales_7d_avg ,
sum(total_sales) over(partition by extract(month from date) order by date) as month_to_date
from temp 
where date > '2014-01-01'::date and date <='2014-02-05'::date
    """
}

### Q4

__Compute the yearly total value of sales for New York, California and Washington, along with % variation in relation to the previous year.__

You must provide a single query, containing the following columns:
* shipping_state
* year
* total_sales, containing the total for such state and year
* variation, containg the % variation in relation to the previous year, i.e. if the yerly total for a state has doubled this field would contain a value of 100.0 . For the first year, this field should be NULL.

**Hint:**
* Take a look at the `lag()` function in the [WINDOW](https://www.postgresql.org/docs/9.5/static/functions-window.html) documentation for computing variation.

In [117]:
%%sql
-- Complete your query here
with temp  as (
    select shipping_state, year,total_sales,lag(total_sales, 1) over (partition by shipping_state order by year)
    from (select shipping_state, extract(year from transaction_datetime) as year, sum(item_price) as total_sales
          from fact_sales
          group by year,shipping_state) as temp1)
select shipping_state,year,total_sales, round(cast((total_sales-lag)/lag*100 as numeric),2) as variation from temp


,shipping_state,year,total_sales,variation
0,Alabama,2014,12264.3662,None
1,Alabama,2015,13737.4424,12.01
2,Alabama,2016,12559.5156,-8.57
3,Alabama,2017,13168.8906,4.85
4,Alaska,2014,16083.7354,None
5,Alaska,2015,14137.499,-12.10
6,Alaska,2016,14052.1855,-0.60
7,Alaska,2017,14404.7402,2.51
8,Arizona,2014,13090.748,None
9,Arizona,2015,12604.5273,-3.71


In [137]:
# submit query in string format
q4 = {
    "query": """
    with temp  as (
    select shipping_state, year,total_sales,lag(total_sales, 1) over (partition by shipping_state order by year)
    from (select shipping_state, extract(year from transaction_datetime) as year, sum(item_price) as total_sales
          from fact_sales
          group by year,shipping_state) as temp1)
select shipping_state,year,total_sales, round(cast((total_sales-lag)/lag*100 as numeric),2) as variation from temp

    """
}

### Q5

__The store has decided to award customers that spent at least $2000 in a single year. Show which customers will receive an award, along with the datetime of the transaction that made each of them pass the minimum value to get the prize.__

You must provide a single query, containing the following columns:
* year
* customer_id
* award_transaction_datetime


In [132]:
%%sql
-- Complete your query here
with temp as (select extract(year from transaction_datetime) as year,
customer_id ,transaction_datetime, sum(item_price) over (partition by customer_id,extract(year from transaction_datetime) order by transaction_datetime) as cumsum
from fact_sales)

select year, customer_id, min(transaction_datetime) as award_transaction_datetime
from temp
where cumsum > 2000
group by year, customer_id

,year,customer_id,award_transaction_datetime
0,2017,300000595,2017-11-24 03:14:36
1,2015,100000876,2015-12-31 17:21:22
2,2015,100000760,2015-11-01 10:57:53
3,2015,400000272,2015-12-24 11:33:27
4,2016,100000282,2016-11-19 18:03:22
5,2017,100000159,2017-11-12 05:46:44
6,2016,100000320,2016-12-15 05:09:19
7,2016,200000549,2016-12-17 01:38:00
8,2014,100000773,2014-12-16 02:36:55
9,2014,400000602,2014-11-19 12:42:58


In [138]:
# submit query in string format
q5 = {
    "query": """
    with temp as (select extract(year from transaction_datetime) as year,
customer_id ,transaction_datetime, sum(item_price) over (partition by customer_id,extract(year from transaction_datetime) order by transaction_datetime) as cumsum
from fact_sales)

select year, customer_id, min(transaction_datetime) as award_transaction_datetime
from temp
where cumsum > 2000
group by year, customer_id
    """
}

## Submission

### Create your submission file¶

Run the following cell to create a results file for your homework

DO NOT MODIFY THE FOLLOWING CELL!!

In [ ]:
import datetime
import json

script_path = '{0}/{1}/fs/Instabase%20Drive'.format(USER, REPO)

with ib.open('results'.format(script_path), "w") as f:
    result = dict(
        q1=q1,
        q2=q2,
        q3=q3,
        q4=q4,
        q5=q5,
        uni=UNI,
        user=USER
    )
    f.write(json.dumps(result))
    print "UNI is {0}".format(UNI)
    print "Result file created at: {0}".format(datetime.datetime.now())
  
    print
    print "Check your results: http://instabase.com/{0}/AA4/results".format(script_path)

UNI is wt2271
Result file created at: 2018-11-13 04:55:19.798290

Check your results: http://instabase.com/wesley/my-repo/fs/Instabase%20Drive/AA4/results


Finally, Submit your __entire AA4 folder including your notebook and results file__ at the following URL:
    
https://www.instabase.com/apps/file-submission/cmd/submit/9eb6cb68-afdd-4042-a94b-1176521ccdbb
